<a href="https://colab.research.google.com/github/Jignacio14/7506R-1C2023-GRUPO09/blob/Pruebas/7506R_TP1_GRUPO9_CHP1_ENTREGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pruebas

En este apartado se estara trabajando el analisis exploratorio de los datos que se estaran usando para el trabajo practico #1

In [29]:
import pandas as pd 
import numpy as np
import sklearn as sk
import seaborn as sns
import matplotlib as mpl

In [30]:
hotelsdf = pd.read_csv("./hotels_train.csv")

print("El data frame esta compuesto por "f"{hotelsdf.shape[0]}"" filas y "f"{hotelsdf.shape[1]}"" columnas")

El data frame esta compuesto por 61913 filas y 33 columnas


Un vistaso básico a la información contenida en el dataframe viene dada por:

In [31]:
pd.concat([hotelsdf.head(2), hotelsdf.sample(5), hotelsdf.tail(2)])

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,id,is_canceled
0,City Hotel,49,2016,September,37,5,1,2,1,0.0,...,NaN,0,Transient,115.50,0,1,Canceled,2016-08-25,7aa4cc6b-b92c-4061-b21d-4f9e0cac4689,1
1,Resort Hotel,4,2015,October,44,31,0,1,2,0.0,...,NaN,0,Transient,42.00,1,0,Check-Out,2015-11-01,b1428f80-c56c-4ae4-91a9-6962edae08b1,0
10866,City Hotel,429,2016,September,39,23,0,2,2,0.0,...,NaN,330,Transient-Party,65.00,0,0,Check-Out,2016-09-25,f00d632f-6751-4a27-a44a-76a85d9ee54f,0
11261,Resort Hotel,0,2016,April,17,18,1,0,1,0.0,...,NaN,0,Transient,52.00,1,0,Check-Out,2016-04-19,8798c00d-ca8a-4cfb-aae6-9ba15717e7c3,0
29730,City Hotel,272,2016,August,32,5,0,1,2,0.0,...,NaN,41,Transient,65.00,0,0,Canceled,2015-12-18,7f36d57f-72f7-42f6-b848-e54e515a8edf,1
11249,City Hotel,56,2016,May,23,30,1,0,1,0.0,...,NaN,0,Transient-Party,120.00,0,0,Check-Out,2016-05-31,42efef93-48bb-4784-9287-160280e0aa13,0
20819,City Hotel,181,2016,September,39,23,0,2,3,0.0,...,NaN,0,Transient,169.00,0,0,Check-Out,2016-09-25,038b3908-b74c-45ac-bda6-349479f0d2a4,0
61911,Resort Hotel,143,2015,October,41,10,2,5,2,0.0,...,NaN,0,Transient,36.05,0,0,Canceled,2015-07-08,01906859-1094-4e37-b13e-b015b594d159,1
61912,City Hotel,57,2016,April,17,17,2,5,2,0.0,...,NaN,0,Transient,143.29,0,0,Canceled,2016-04-09,7137e599-4442-41fb-b147-4aba9f344e38,1


Por otro lado un vistazo un poco más detallado a la informacion contenida por el dataframe puede verse:

In [10]:
hotelsdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61913 entries, 0 to 61912
Data columns (total 33 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           61913 non-null  object 
 1   lead_time                       61913 non-null  int64  
 2   arrival_date_year               61913 non-null  int64  
 3   arrival_date_month              61913 non-null  object 
 4   arrival_date_week_number        61913 non-null  int64  
 5   arrival_date_day_of_month       61913 non-null  int64  
 6   stays_in_weekend_nights         61913 non-null  int64  
 7   stays_in_week_nights            61913 non-null  int64  
 8   adults                          61913 non-null  int64  
 9   children                        61909 non-null  float64
 10  babies                          61913 non-null  int64  
 11  meal                            61913 non-null  object 
 12  country                         

Segun la clasificacion de las variables podemos establecer lo siguiente sobre los datos extraidos del dataframe

Variables cuantitativas, entre las cuales podemos encontrar:

· arrival_date_year

· arrival_date_month 

· arrival_date_week_number  
  
· arrival_date_day_of_month

· stays_in_weekend_nights

· days_in_waiting_list

· stays_in_week_nights

· required_car_parking_spaces

continuar...

In [11]:
hotelsdf.isnull().sum()

hotel                                 0
lead_time                             0
arrival_date_year                     0
arrival_date_month                    0
arrival_date_week_number              0
arrival_date_day_of_month             0
stays_in_weekend_nights               0
stays_in_week_nights                  0
adults                                0
children                              4
babies                                0
meal                                  0
country                             221
market_segment                        0
distribution_channel                  0
is_repeated_guest                     0
previous_cancellations                0
previous_bookings_not_canceled        0
reserved_room_type                    0
assigned_room_type                    0
booking_changes                       0
deposit_type                          0
agent                              7890
company                           58761
days_in_waiting_list                  0


In [12]:
hotelsdf['hotel'].value_counts().index.tolist()

['City Hotel', 'Resort Hotel']